## Business Problem:
+ New property owners face a challenge in determining an optimal pricing strategy that maintains customer ratings while ensuring consistent revenue generation.

## Business Objective:
+ The objective of this analysis is to help new property owners determine an appropriate pricing strategy that maintains good customer ratings while ensuring steady revenue generation.

## Problem Statement:
+ New property owners often struggle to set the right price for their villas and apartments, as incorrect pricing can negatively affect customer ratings or reduce revenue. This analysis aims to study the relationship between pricing and ratings to support better pricing decisions.New property owners often struggle to set the right price for their villas and apartments, as incorrect pricing can negatively affect customer ratings or reduce revenue. This analysis aims to study the relationship between pricing and ratings to support better pricing decisions.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
url = 'https://www.booking.com/searchresults.en-gb.html?label=gen173nr-10CAQoggJCCnNlYXJjaF9nb2FICVgEaGyIAQGYATO4ARfIAQzYAQPoAQH4AQGIAgGoAgG4Av2k8soGwAIB0gIkNDhjODg5YTAtZDAxMC00YTgxLWE5NmEtYTIwY2U1MDQ3YTJh2AIB4AIB&aid=304142&ss=Goa&ssne=Goa&ssne_untouched=Goa&efdco=1&lang=en-gb&src=searchresults&dest_id=4127&dest_type=region&checkin=2026-03-17&checkout=2026-03-19&group_adults=2&no_rooms=1&group_children=0&nflt=ht_id%3D201%3Bht_id%3D213'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

In [4]:
wait = WebDriverWait(driver, 20)

#  Scroll till bottom
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

#  LOAD MORE RESULTS (MULTIPLE CLICKS)
for i in range(20): 
    try:
        load_more_button = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[.//span[text()='Load more results']]")
            )
        )
        driver.execute_script("arguments[0].click();", load_more_button)
        time.sleep(4)  
    except:
        print("Load more button not found")
        break

In [5]:
property_name = []
location = []
property_type = []
room_type = []
price = []
rating = []
review_count = []
free_cancellation = []


elements = driver.find_elements(By.CSS_SELECTOR, ".b87c397a13.a3e0b4ffd1")

for el in elements:
    text = el.text
    property_name.append(text)
    

elements = driver.find_elements(By.CSS_SELECTOR, ".d823fbbeed.f9b3563dd4")

for el in elements:
    text = el.text
    if text != "Show on map":
        location.append(text)


elements = driver.find_elements(By.CLASS_NAME, "a0f53222cd")

for el in elements:
    text = el.text
    if ("Entire" in text) or ("Private suite" in text):
        property_type.append(text)


elements = driver.find_elements(By.CSS_SELECTOR, ".fff1944c52.f254df5361")

for el in elements:
    text = el.text.split(",")[0]
    room_type.append(text)


elements = driver.find_elements(By.CSS_SELECTOR, ".b87c397a13.f2f358d1de.ab607752a2")

for el in elements:
    text = el.text.replace("₹ ", "").replace(",", "")
    price.append(text)


cards = driver.find_elements(By.XPATH, "//div[@data-testid='property-card']")

for card in cards:
    try:
        r = card.find_element(By.CSS_SELECTOR, ".f63b14ab7a.dff2e52086").text
        rating.append(r)
    except:
        rating.append(None)


cards = driver.find_elements(By.XPATH, "//div[@data-testid='property-card']")

for card in cards:
    try:
        r = card.find_element(By.CSS_SELECTOR, ".fff1944c52.fb14de7f14.eaa8455879").text.split()[0].replace(",", "")
        review_count.append(r)
    except:
        review_count.append(None)


elements = driver.find_elements(By.XPATH, "//div[@data-testid='property-card']")

for el in elements:
    try:
        text = el.text.lower()
        if "free cancellation" in text:
            free_cancellation.append("Yes")
        else:
            free_cancellation.append("No")
    except:
        free_cancellation.append("No")

In [6]:
dic = {'Property_Name':property_name,'Location':location,'Property_Type':property_type,'Room_Type':room_type,'Price':price,'Rating':rating,'Reviews':review_count,'Free_Cancellation':free_cancellation}

In [7]:
df = pd.DataFrame(dic)

In [8]:
df

,Property_Name,Location,Property_Type,Room_Type,Price,Rating,Reviews,Free_Cancellation
0,Ronne's Candy Floss Blue Lagoon villa,Guirim,Entire villa,Three-Bedroom Villa,13159,9.5,4,No
1,"Nature's Nest Eco Resort Goa, Near Dudhsagar W...",Molem,Entire villa,Standard Villa,19800,9.5,70,Yes
2,Caravela Beach Resort,Varca,Entire villa,2 Bed Room Garden Villa,85680,8.6,1719,No
3,The St. Regis Goa Resort,Cavelossim,Entire villa,John Jacob Astor Villa - 2 Bedroom Villa,476000,8.9,420,Yes
4,Aveeno Resort near Calangute and Baga,Goa,Entire villa,Standard Villa,4248,5.2,44,Yes
...,...,...,...,...,...,...,...,...
553,Villa Poorvi by Hireavilla - 4BHK Villa in Sio...,Marna,Entire villa,Villa,54243,None,None,Yes
554,'Golden Bay' 2bhk beach villa,Benaulim,Entire villa,Villa,16000,10,1,Yes
555,1 BHK near Calangute Beach A1-201,Calangute,Entire apartment,One-Bedroom Apartment with Balcony,5000,None,None,Yes
556,"Love Story Villas, Baga-Private Pool Villas",Arpora,Entire villa,Three-Bedroom Villa,25200,10,3,Yes


In [9]:
df.to_csv(r'Raw Dataset.csv')